# Recurrent Neural Networks
You should build an end-to-end machine learning pipeline using a recurrent neural network model. In particular, you should do the following:
- Load the `jena climate` dataset using [Pandas](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html). You can find this dataset in the [keras repository](https://keras.io/examples/timeseries/timeseries_weather_forecasting/).
- Split the dataset into training, validation, and test sets. Note that you cannot split time series using [Scikit-Learn](https://keras.io/examples/timeseries/timeseries_weather_forecasting/).
- Build an end-to-end machine learning pipeline, including a [recurrent neural network](https://keras.io/examples/timeseries/timeseries_weather_forecasting/) model.
- Optimize your pipeline by validating your design decisions.
- Test the best pipeline on the test set and report various [evaluation metrics](https://scikit-learn.org/0.15/modules/model_evaluation.html).  
- Check the documentation to identify the most important hyperparameters, attributes, and methods of the model. Use them in practice.

Data Loading, Sampling, and Target Creation

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import mean_squared_error

In [4]:
import zipfile

uri = "https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip"
zip_path = keras.utils.get_file(origin=uri, fname="jena_climate_2009_2016.csv.zip")

csv_file_name = "jena_climate_2009_2016.csv"
try:
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extract(csv_file_name)
    df = pd.read_csv(csv_file_name)
except FileNotFoundError:
    print(f"Error: The file '{csv_file_name}' was not found after extraction.")
    df = None
except Exception as e:
    print(f"An unexpected error occurred during file extraction or reading: {e}")
    df = None
if df is not None:
    df = df.iloc[5::6].reset_index(drop=True)

    df['Date Time'] = pd.to_datetime(df['Date Time'], format='%d.%m.%Y %H:%M:%S')

    df = df[df['Date Time'].dt.hour == 1].reset_index(drop=True)

    feature_cols = ['T (degC)', 'p (mbar)', 'rh (%)', 'wv (m/s)', 'VPact (mbar)']
    data = df[feature_cols].copy()

    data['Target_T'] = data['T (degC)'].shift(-1)

    data.dropna(inplace=True)

    X = data[feature_cols].values
    Y = data['Target_T'].values
else:

    print("DataFrame 'df' could not be loaded. Subsequent data processing steps have been skipped.")

Data Splitting and Normalization

In [5]:
train_split_pct = 0.7
val_split_pct = 0.15
test_split_pct = 0.15

N = X.shape[0]
train_end = int(train_split_pct * N)
val_end = int((train_split_pct + val_split_pct) * N)

X_train_raw, Y_train = X[:train_end], Y[:train_end]
X_val_raw, Y_val = X[train_end:val_end], Y[train_end:val_end]
X_test_raw, Y_test = X[val_end:], Y[val_end:]

train_mean = X_train_raw.mean(axis=0)
train_std = X_train_raw.std(axis=0)

X_train = (X_train_raw - train_mean) / train_std
X_val = (X_val_raw - train_mean) / train_std
X_test = (X_test_raw - train_mean) / train_std

Sequence Matrix Creation

In [6]:
SEQUENCE_LENGTH = 5
BATCH_SIZE = 64

def create_time_series_dataset(features, targets, seq_len, batch_size):
    feature_tensor = tf.constant(features, dtype=tf.float32)
    target_tensor = tf.constant(targets, dtype=tf.float32)

    data = keras.utils.timeseries_dataset_from_array(
        data=feature_tensor,
        targets=target_tensor[seq_len:],
        sequence_length=seq_len,
        sequence_stride=1,
        sampling_rate=1,
        batch_size=batch_size
    )
    return data

train_dataset = create_time_series_dataset(X_train, Y_train, SEQUENCE_LENGTH, BATCH_SIZE)
val_dataset = create_time_series_dataset(X_val, Y_val, SEQUENCE_LENGTH, BATCH_SIZE)
test_dataset = create_time_series_dataset(X_test, Y_test, SEQUENCE_LENGTH, BATCH_SIZE)

for batch in train_dataset.take(1):
    inputs, targets = batch
    INPUT_SHAPE = (inputs.shape[1], inputs.shape[2])

Model Building, Training, and Evaluation

In [8]:
LEARNING_RATE = 0.001
EPOCHS = 20
LSTM_UNITS = 32

model = keras.Sequential([
    keras.layers.GRU(LSTM_UNITS, input_shape=INPUT_SHAPE),
    keras.layers.Dense(1)
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss="mse"
)

es_callback = keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=val_dataset,
    callbacks=[es_callback],
    verbose=0
)

print(" Model Training Complete")

print("Evaluating model on the test set...")
test_loss = model.evaluate(test_dataset, verbose=0)
print(f"Test Mean Squared Error (MSE): {test_loss:.4f}")

sample_input, sample_target = next(iter(test_dataset))
prediction = model.predict(sample_input, verbose=0)[0]
actual = sample_target.numpy()[0]

print("\n Example Prediction")
print(f"Predicted Next Day Temp (normalized): {prediction[0]:.4f}")
print(f"Actual Next Day Temp (normalized):   {actual:.4f}")

print("\n Hyperparameter & Method Check ")
print(f"Hyperparameter (GRU Units): {LSTM_UNITS}")
print(f"Hyperparameter (Sequence Length): {SEQUENCE_LENGTH}")
print(f"Method Used: model.fit() (Training)")
print(f"Method Used: model.evaluate() (Testing)")
print(f"Attribute Check (Total Parameters): {model.count_params()}")

 Model Training Complete
Evaluating model on the test set...
Test Mean Squared Error (MSE): 16.6793

 Example Prediction
Predicted Next Day Temp (normalized): 7.0987
Actual Next Day Temp (normalized):   10.0000

 Hyperparameter & Method Check 
Hyperparameter (GRU Units): 32
Hyperparameter (Sequence Length): 5
Method Used: model.fit() (Training)
Method Used: model.evaluate() (Testing)
Attribute Check (Total Parameters): 3777
